<a href="https://colab.research.google.com/github/iampundir/Text-Analytics/blob/main/Working_with_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/41q6tiwgqxy3r95/The%20Maze%20Runner.csv?dl=0

--2020-10-12 17:36:05--  https://www.dropbox.com/s/41q6tiwgqxy3r95/The%20Maze%20Runner.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/41q6tiwgqxy3r95/The%20Maze%20Runner.csv [following]
--2020-10-12 17:36:05--  https://www.dropbox.com/s/raw/41q6tiwgqxy3r95/The%20Maze%20Runner.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb2d0150efbe6f1608711c1f405.dl.dropboxusercontent.com/cd/0/inline/BBJncMq8QES0Waex59NdLOKyAkUAFZEqVAqJP-YUrcX6emlNVvtwVkHX2WjXqJxYVzTvwDJv8kurIjmdzYg38Wya03QiOgQTAtTEbIgtWtJvag/file# [following]
--2020-10-12 17:36:05--  https://ucb2d0150efbe6f1608711c1f405.dl.dropboxusercontent.com/cd/0/inline/BBJncMq8QES0Waex59NdLOKyAkUAFZEqVAqJP-YUrcX6emlNVvtwVkHX2WjXqJxYVzTvwDJv8kurIjmdzYg38Wya03QiOg

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
reviews = pd.read_csv('/content/The Maze Runner.csv?dl=0', index_col=0)

In [4]:
reviews.head()

,Ratings,Reviews
0,1/10,"My English is so bad, I don't speak English, ..."
1,1/10,The trailer got me into the cinema and it was...
2,4/10,I went into this film with low expectations. ...
3,3/10,While I was happy to finally see a male-orien...
4,7/10,It's rare for a big movie studio to trust the...


In [5]:
import re

In [6]:
ratings = [re.sub("/10","",str(x)) for x in reviews.Ratings]

In [7]:
reviews['Ratings'] = ratings

In [8]:
reviews.Ratings = reviews.Ratings.astype('float')

In [9]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ratings  481 non-null    float64
 1   Reviews  500 non-null    object 
dtypes: float64(1), object(1)
memory usage: 11.7+ KB


In [10]:
from wordcloud import STOPWORDS

In [11]:
tfidf = TfidfVectorizer(stop_words=STOPWORDS, token_pattern="[a-z']+")

In [12]:
tfidf.fit(reviews.Reviews)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'all', 'also', 'am', 'an', 'and', 'any', 'are',
                            "aren't", 'as', 'at', 'be', 'because', 'been',
                            'before', 'being', 'below', 'between', 'both',
                            'but', 'by', 'can', "can't", 'cannot', 'com', ...},
                strip_accents=None, sublinear_tf=False, token_pattern="[a-z']+",
                tokenizer=None, use_idf=True, vocabulary=None)

In [13]:
tfidf_mat = tfidf.transform(reviews.Reviews)

In [14]:
tfidf_mat

<500x8420 sparse matrix of type '<class 'numpy.float64'>'
	with 56045 stored elements in Compressed Sparse Row format>

In [15]:
query = "James Dashner's book and hunger games"

In [16]:
query_tfidf_mat = tfidf.transform([query])

In [17]:
query_tfidf_mat

<1x8420 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [18]:
similarity = pairwise_distances(X=tfidf_mat, Y=query_tfidf_mat, metric='cosine')
np.argsort(similarity.flatten())[:5]

array([261, 487, 377, 497, 336])

In [19]:
reviews.loc[487,'Reviews']

" 'The Maze Runner, based on the first part of James Dashner's young-adult post-apocalyptic sci-fi trilogy, is a good and interesting start to the series. It has enough thrills and twists to keep you intrigued and on-the-edge till the end. The cast, which is almost entirely filled with teenagers, has done quite a commendable job and help making the dystopian scenario quite believable.The plot may seem like one of those stories you've heard time and again, but it does have new and interesting things to offer. Since there are so many characters, their development is quite limited. Few of the answers regarding the plot are answered towards the end; maybe, the remaining answers are left for the sequels.I would have no issues if this becomes a big movie franchise like 'The Hunger Games' simply because it's much better than other big movie franchises of the past, like The Twilight Series. "

In [20]:
class tfidf_mat_creator(BaseEstimator, TransformerMixin):
  def __init__(self, tfidf):
    self.tfidf = tfidf

  def fit(self,X,y=None):
    return self
  
  def transform(self, X, y=None):
    return self.tfidf.transform([X])

class index_most_similar(BaseEstimator, TransformerMixin):
  def __init__(self, tfidf_mat, metric='euclidean', k=5):
    self.tfidf_mat = tfidf_mat
    self.metric = metric
    self.k = k

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    return np.argsort(pairwise_distances(X=X, Y=self.tfidf_mat, metric=self.metric)).flatten()[:self.k]

In [21]:
pipeline = Pipeline([('conv_to_tfidf',tfidf_mat_creator(tfidf=tfidf)),
                     ('extract_similarity',index_most_similar(tfidf_mat=tfidf_mat))])

In [22]:
pipeline.fit(query)

Pipeline(memory=None,
         steps=[('conv_to_tfidf',
                 tfidf_mat_creator(tfidf=TfidfVectorizer(analyzer='word',
                                                         binary=False,
                                                         decode_error='strict',
                                                         dtype=<class 'numpy.float64'>,
                                                         encoding='utf-8',
                                                         input='content',
                                                         lowercase=True,
                                                         max_df=1.0,
                                                         max_features=None,
                                                         min_df=1,
                                                         ngram_range=(1, 1),
                                                         norm='l2',
                                                    

In [23]:
pipeline.transform(query)

array([261, 487, 377, 497, 336])